# Project 2


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="https://github.com/andresmarin94/Repositorio_Personal/blob/master/moviegenre.png?raw=1"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4200-20191-p2-moviegenreclassification/overview


### Data

Input:
- movie plot

Output:
Probability of the movie belong to each genre


### Evaluation

- 20% API
- 30% Create a solution using with a Machine Learning algorithm - Presentation (5 slides)
- 50% Performance in the Kaggle competition (Normalized acording to class performance in the private leaderboard)


### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

## Sample Submission

In [36]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
import xgboost
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
import re

In [38]:
dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

### Create count vectorizer


In [39]:
dataTraining.head(5)

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [40]:
dataTesting.head(5)

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [41]:
dataTraining['title']=dataTraining['title'].str.lower()
dataTraining['title']=dataTraining['title'].str.replace(',', '') 
dataTraining['title']=dataTraining['title'].str.replace('.', '') 
dataTraining['title']=dataTraining['title'].str.replace('-', '') 
dataTraining['title']=dataTraining['title'].str.replace('_', '') 
dataTraining['plot']=dataTraining['plot'].str.lower()
dataTraining['plot']=dataTraining['plot'].str.replace(',', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('.', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('-', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('_', '') 

In [42]:
dataTesting['plot']=dataTesting['plot'].str.lower()
dataTesting['plot']=dataTesting['plot'].str.replace(',', '') 
dataTesting['plot']=dataTesting['plot'].str.replace('.', '') 
dataTesting['plot']=dataTesting['plot'].str.replace('-', '') 
dataTesting['plot']=dataTesting['plot'].str.replace('_', '')
dataTesting['title']=dataTesting['title'].str.lower()
dataTesting['title']=dataTesting['title'].str.replace(',', '') 
dataTesting['title']=dataTesting['title'].str.replace('.', '') 
dataTesting['title']=dataTesting['title'].str.replace('-', '') 
dataTesting['title']=dataTesting['title'].str.replace('_', '') 

In [43]:
wordnet_lemmatizer = WordNetLemmatizer()

In [44]:
dataTraining['uno']= dataTraining['title']+dataTraining['plot']
dataTesting['uno'] = dataTesting['title']+dataTesting['plot']

data = dataTraining['uno'].append(dataTesting['uno'])
years = dataTraining['year'].append(dataTesting['year'])

In [45]:
wordnet_lemmatizer = WordNetLemmatizer()
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word, 's') for word in words]

vect = TfidfVectorizer(analyzer=split_into_lemmas, min_df=2,max_features=None,sublinear_tf=True,
                       strip_accents='unicode',ngram_range=(1,2),stop_words='english')

#X_dtm = vect.fit_transform(dataTraining['uno'],dataTraining['year'])
#X_dtm.shape
X_dtm = vect.fit_transform(data, years)

In [46]:
X_dtm_test = X_dtm[dataTraining.shape[0]:,:]
X_dtm = X_dtm[:dataTraining.shape[0],:]

### Create y

In [47]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [48]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.3, random_state=666)

### Train multi-class multi-label model

In [49]:

#clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1,max_depth=10,n_estimators=1000, random_state=60,criterion='entropy'))

In [50]:
LogReg_Model = OneVsRestClassifier(LogisticRegression(multi_class='multinomial',solver='newton-cg',C=1.5,
#                                                     warm_start=True,penalty='elasticnet',random_state=456, cv=10))
                                                      warm_start=True, penalty='l2',random_state=999), n_jobs=-1)

LogReg_Model.fit(X_dtm, y_genres)
y_pred_genres = LogReg_Model.predict_proba(X_test)
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.9967011998269161

### Predict the testing dataset

In [51]:


cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = LogReg_Model.predict_proba(X_dtm_test)


In [52]:
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)

In [53]:
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.134051,0.073878,0.025038,0.013649,0.345190,0.091437,0.007755,0.389886,0.042179,0.124742,...,0.041198,0.059782,0.000795,0.744693,0.021727,0.007269,0.011798,0.111841,0.013360,0.019492
4,0.130505,0.025498,0.023197,0.120260,0.236176,0.353956,0.034612,0.880085,0.031787,0.016500,...,0.020940,0.038048,0.001103,0.049325,0.011523,0.010007,0.019921,0.395967,0.030852,0.012784
5,0.050278,0.012661,0.003861,0.054951,0.052461,0.754219,0.011673,0.807502,0.006741,0.016954,...,0.013003,0.473465,0.001042,0.128824,0.025417,0.003851,0.018768,0.601765,0.019186,0.009530
6,0.086433,0.085906,0.008635,0.042006,0.130607,0.044529,0.007448,0.834776,0.024779,0.038127,...,0.017634,0.091954,0.000812,0.148411,0.118043,0.004799,0.022458,0.409718,0.076917,0.018982
7,0.022749,0.030630,0.017901,0.024319,0.160061,0.065459,0.013257,0.377242,0.030876,0.109734,...,0.015774,0.075222,0.000834,0.133747,0.425681,0.008434,0.007656,0.344601,0.009135,0.012498


In [54]:
res.to_csv('testtest.csv', index_label='ID')